In [9]:
import os, datetime
from fasp.search import DiscoverySearchClient
from fasp.loc import EGAFileClient
from fasp.runner import FASPRunner

#from fasp.workflow import DNAStackWESClient
#from fasp.workflow import GCPLSsamtools

faspRunner = FASPRunner(pauseSecs=0)
creditor = faspRunner.creditor
settings = faspRunner.settings

# Step 1 - Discovery
# query for relevant files
searchClient = DiscoverySearchClient('https://ga4gh-search-adapter-presto-public.prod.dnastack.com/')
query = '''SELECT sample_submitter_id, fileid, filename 
FROM dbgap_demo.scr_ega.scr_egapancreatic_sample_multi p join dbgap_demo.scr_ega.scr_egapancreatic_files f on f.sample_primary_id = p.sample_primary_id 
where phenotype = 'pancreatic adenocarcinoma' limit 3'''
query_job = searchClient.runQuery(query)

# Step 2 - Use htsget at EGA
fileClient = EGAFileClient('~/.keys/ega.credentials')

# Step 3 - set up a class that run a compute for us
location = 'projects/{}/locations/{}'.format(settings['GCPProject'], settings['GCPPipelineRegion'])
#wesClient = GCPLSsamtools(location, settings['GCPOutputBucket'])

Running <ipython-input-9-132c77837b4d>
_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________


The EGA dataset used above does not contains index files so the htsget call cannot be used. These lines are commented out.

The example does show a simulated DRS response generated using the pyega3 module

In [11]:
# repeat steps 2 and 3 for each row of the query
for row in query_job:

    print("sample={}, EGAFileID={}".format(row[0], row[1]))

    # Step 2 - Get a simulated DRS response
    drsResponse = fileClient.getObject(row[1])
    fileSize = drsResponse['size']
    print(drsResponse)

    print("File size:{}".format(fileSize))
    localfile = row[2]
    #htsgetClient.htsget(row[1], 'chr1', 100000, 102000, 'BAM', localfile)

    # Step 3 - Run a pipeline on the file 
    outfile = "{}.txt".format(row[0])
    #pipeline_id = wesClient.runWorkflow(localfile, outfile)
    #print('submitted:{}'.format(pipeline_id))

    via = 'lsapi'
    note = 'samtools on htsget BAM'

    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    #faspRunner.logRun(time, via, note,  pipeline_id, outfile, str(fileSize),
    #    searchClient, htsgetClient, wesClient)
    print('_'*80)



sample=Pam02_LiM7, EGAFileID=EGAF00001554155
{'access_methods': [{'access_id': 'ega', 'region': '', 'type': 'pyega3'}], 'checksums': [{'checksum': '9ce6eee848cfff52b73ae341bdadc1c5', 'type': 'md5'}], 'created_time': '2020-02-26T18:01:43.252269', 'id': 'EGAF00001554155', 'mime_type': 'application/json', 'description': 'EGAR00001523154/PAM02/SS6004350.bam.cip', 'name': 'SS6004350.bam', 'self_uri': 'drs://egasim/EGAF00001554155', 'size': 199417133437, 'updated_time': '2020-02-26T18:01:43.252275'}
File size:199417133437
________________________________________________________________________________
sample=Pam02_LiM8, EGAFileID=EGAF00001554156
{'access_methods': [{'access_id': 'ega', 'region': '', 'type': 'pyega3'}], 'checksums': [{'checksum': '72f414535f5f4c2fdcd1d34cb55bb9ef', 'type': 'md5'}], 'created_time': '2020-02-26T18:01:43.252269', 'id': 'EGAF00001554156', 'mime_type': 'application/json', 'description': 'EGAR00001523155/PAM02/SS6004351.bam.cip', 'name': 'SS6004351.bam', 'self_uri'